# Feature Engineering

COmbine all of the feature calculations into a single notebook and output a full featue set as a dataframe.

### Import Libraries

In [ ]:
#import required libraries
from utils import aws # used to create aws session and load parquet 
import pandas as pd
import numpy as np
import ast 
import dask.dataframe as dd

### Load LOB Data

Currently this is run using the sample dataset, but going forward will need to be run using the full LOB dataset.

In [ ]:
#load sample lob from s3 to a dask dataframe
samp_lob_ddf = aws.load_s3_file_as_ddf("s3://dsmp-ol2/processed-data/lob_sample_data.parquet")

In [ ]:
#compute the dask datafram to a pandas dataframe
df = samp_lob_ddf.compute()

In [ ]:
df.head()

### Preprocessing

In [ ]:
#convert string to lists
df['Bid'] = df['Bid'].apply(ast.literal_eval)
df['Ask'] = df['Ask'].apply(ast.literal_eval)

In [ ]:
#drop missing rows in mid price
df = df.dropna(subset=['Mid_Price']) # This assumes the number of missing mid prices is small
# TODO following EDA determine if this is the correct decision

## Feature Engineering

## Simple Financial Features

### Volume and OBV

- OBV -on balance volume- confirm trends by analysing volume changes. Persistnet rising obv suggests accumulation / buying pressure- associated with upward price movement. Persistent falling obv suggests distribution /selling pressure. Theory is that changes in volume preced price movements. Increase in volume often precedes a change in price direction. Use as trend,reversal and breakout confirmation.


- If today's close is higher than yesterday's close, then OBV = Previous OBV + today's volume.
- If today's close is lower than yesterday's close, then OBV = Previous OBV - today's volume.
- If today's close is equal to yesterday's close, then OBV = Previous OBV.

In [ ]:
#calculate order volume at each timestep
df['Volume'] = df['Bid'].apply(lambda x: sum([qty for price, qty in x])) + \
               df['Ask'].apply(lambda x: sum([qty for price, qty in x]))

#calc obv
#should be done using close but we can approximate with mid but might not be as reliable
direction = df['Mid_Price'].diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
df['OBV'] = (direction * df['Volume']).cumsum()

### Order Book Imbalance -- IGNORE as Using Volume Imbalance

Order book imbalance reflects the proportion of buy to sell orders and can indicate potential price movements based on supply and demand dynamics.

calc as:

$$
\text{Imbalance} = \frac{Q_{\text{bid}} - Q_{\text{ask}}}{Q_{\text{bid}} + Q_{\text{ask}}}
$$

where:
- $Q_{\text{bid}}$ is the total quantity of buy orders,
- $Q_{\text{ask}}$ is the total quantity of sell orders.

Positive imbalance suggests a predominance of buy orders, which could indicate upward pressure on prices, while a negative imbalance suggests the opposite.

In [ ]:
# #This is the same as Volume Imbalance. Volume imbalance is clearer.
# #calc total quantity
# def total_quantity(price_qty_list):
#     if not price_qty_list:
#         return 0
#     total_qty = sum(qty for _, qty in price_qty_list)
#     return total_qty

# #calc total bid ask quantities
# df['Total_Bid_Qty'] = df['Bid'].apply(total_quantity)
# df['Total_Ask_Qty'] = df['Ask'].apply(total_quantity)

# #calc order book imbalance
# df['Order_Book_Imbalance'] = (df['Total_Bid_Qty'] - df['Total_Ask_Qty']) / (df['Total_Bid_Qty'] + df['Total_Ask_Qty'])

### Volume Imbalance

Volume Imbalance is a metric used to assess the balance between supply and demand in a market at any given time, based on order book data. It compares the total volume of buy orders (bids) to the total volume of sell orders (asks), providing insight into potential price movements.

**Formula**

The Volume Imbalance ($VI$) is calculated using the formula:

$$ VI = \frac{V_{\text{bids}} - V_{\text{asks}}}{V_{\text{bids}} + V_{\text{asks}}} $$

where:
- $V_{\text{bids}}$ is the total volume of all bid orders.
- $V_{\text{asks}}$ is the total volume of all ask orders.

**Interpretation**

- $VI$ ranges from -1 to 1.
- A $VI$ closer to 1 indicates a higher volume of bids relative to asks, suggesting upward pressure on price.
- A $VI$ closer to -1 indicates a higher volume of asks relative to bids, suggesting downward pressure on price.
- A $VI$ around 0 indicates a balance between bid and ask volumes, suggesting a stable market condition without clear directional pressure on price.

In [ ]:
def calculate_volume_imbalance(row):
    #calculate total bid volume
    total_bid_volume = sum([volume for _, volume in row['Bid']])
    
    #calculate total ask volume
    total_ask_volume = sum([volume for _, volume in row['Ask']])
    
    #error handling (div by 0)
    total_volume = total_bid_volume + total_ask_volume
    if total_volume == 0:
        return 0  #return 0 imbalance when there are no bids or asks
    
    #calculate volume imbalance
    volume_imbalance = (total_bid_volume - total_ask_volume) / total_volume
    return volume_imbalance

#apply
df['Total_Volume_Imbalance'] = df.apply(calculate_volume_imbalance, axis=1)

### Weighted Mid-Price by Order Imbalance -- Check this calculation

Given **Mid Price** ($M$) is already calculated, we calculate **Order Imbalance** ($I$) and the **Weighted Mid Price** ($M_W$) as follows:

**Order Imbalance**
With $V_B$ representing the volume of the best bid and $V_A$ representing the volume of the best ask, the order imbalance ($I$) is calculated by the formula:

$$ I = \frac{V_B - V_A}{V_B + V_A} $$

**Weighted Mid Price by Order Imbalance**
We adjust the mid price based on the order imbalance using the formula:

$$ M_W = M \times (1 + k \times I) $$

Here, $k$ is a scaling factor that modulates the effect of the order imbalance on the mid price. This factor can be empirically determined to best reflect the impact of order volume differences on the price.

This approach allows us to adjust the given mid price considering the balance or imbalance between buy and sell orders, reflecting a more nuanced market value.

In [ ]:
# TODO Ask Sam, are Orderbook Imbalance, Order Imbalance and Volume Imbalance the same thing?

In [ ]:
# #first we need to empirically define k

# #future mid-price 
# df['Mid_Price_Future'] = df['Mid_Price'].shift(-1) #TODO Check if this is correct? we will not be able to do this on our test set as we can't look into the future

# #calc price change
# df['Price_Change'] = df['Mid_Price_Future'] - df['Mid_Price']

# # TODO check this is correct. It looks like this is calculating the overall volume not best volume?
# def calculate_order_imbalance(row):
#     if row['Bid'] and row['Ask']:
#         volume_bid = sum([bid[1] for bid in row['Bid']])
#         volume_ask = sum([ask[1] for ask in row['Ask']])
#         return (volume_bid - volume_ask) / (volume_bid + volume_ask)
#     return np.nan

# #calc order imbalance for each row
# df['Order_Imbalance'] = df.apply(calculate_order_imbalance, axis=1)  

In [ ]:
# pip install sklearn

In [ ]:
# #use regression to get value for k
# from sklearn.linear_model import LinearRegression

# #filter out nan
# filtered_df = df.dropna(subset=['Order_Imbalance', 'Price_Change'])

# X = filtered_df[['Order_Imbalance']].values.reshape(-1, 1)
# y = filtered_df['Price_Change'].values

# #fit linear regression model
# model = LinearRegression()
# model.fit(X, y)

# #coefficient of 'Order_Imbalance' as 'k'
# k = model.coef_[0]

# print(f"Derived k value: {k}")

In [ ]:
# #calc weighted mid price col
# import pandas as pd

# def calculate_weighted_mid_price(row, k):
#     #check if nan
#     if pd.isna(row['Mid_Price']):
#         return row['Mid_Price']  #return original if nan
    
#     imbalance = row['Order_Imbalance']
    
#     #adjust mid price based on imbalance
#     weighted_mid_price = row['Mid_Price'] * (1 + k * imbalance)
    
#     return weighted_mid_price

# #apply
# df['Weighted_Mid_Price'] = df.apply(calculate_weighted_mid_price, k=k, axis=1)

### Bid-Ask Spread and Level-1 Data

Bid-ask spread represents the difference between the highest price buyers are willing to pay (**bid price**) and the lowest price sellers are willing to accept (**ask price**). We can use it as a key indicator of market liquidity and efficiency.

We calculate it by:

$$ \text{Bid-Ask Spread} = \text{Ask Price} - \text{Bid Price} $$

where:
- $\text{Ask Price}$ is the lowest price a seller is willing to accept.
- $\text{Bid Price}$ is the highest price a buyer is willing to pay.


A narrower (lower) spread often indicates a more liquid market, where transactions can occur more easily at prices close to the market's consensus value. A wider (higher) spread suggests lower liquidity, higher transaction costs, and potentially more volatility.

In [ ]:
def calculate_level_1(row):
    if not row['Bid'] or not row['Ask']:  # check if either list is empty
        return pd.Series([None, None, None, None, None, None], index=['Bid_Ask_Spread', 'Level_1_Bid_Price', 'Level_1_Ask_Price', 'Level_1_Bid_Quantity', 'Level_1_Ask_Quantity', 'Level_1_Order_Imbalance'])
    
    # extract highest bid price and lowest ask price
    highest_bid_price = row['Bid'][0][0]  # assuming first entry is the highest bid
    lowest_ask_price = row['Ask'][0][0]  # assuming first entry is the lowest ask
    highest_bid_qnt = row['Bid'][0][1]  # assuming first entry is the highest bid
    lowest_ask_qnt = row['Ask'][0][1]  # assuming first entry is the lowest ask
    
    # calculate the spread
    bid_ask_spread = lowest_ask_price - highest_bid_price

    # calculate the level 1 order imbalance
    l1_order_imbalance = highest_bid_qnt - lowest_ask_qnt

    return pd.Series([bid_ask_spread, highest_bid_price, lowest_ask_price, highest_bid_qnt, lowest_ask_qnt, l1_order_imbalance], index=['Bid_Ask_Spread', 'Level_1_Bid_Price', 'Level_1_Ask_Price', 'Level_1_Bid_Quantity', 'Level_1_Ask_Quantity', 'Level_1_Order_Imbalance'])

# apply
df[['Bid_Ask_Spread', 'Level_1_Bid_Price', 'Level_1_Ask_Price', 'Level_1_Bid_Quantity', 'Level_1_Ask_Quantity', 'Level_1_Order_Imbalance']] = df.apply(calculate_level_1, axis=1)

In [ ]:
df

## Complex Financial Features

### RSI

RSI (relative strength index)-popular- overbought and oversold conditions-calculates with avg gain, avg loss and relative strength. You need to check direction not just value to understand trend. 

- above 70- overbought
- below 30- oversold
- 50-70- a strong bullish trend
- 30-50 -a moderate bullish trend
- 50-30- a strong bearish trend
- 70-40- a moderate bearish trend

In [ ]:
#rsi
delta = df['Mid_Price'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=20).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=20).mean()
RS = gain / loss

df['RSI'] = 100 - (100 / (1 + RS))

### Stochastic RSI

The Stochastic RSI is an indicator used to identify overbought and oversold conditions by measuring the level of the RSI relative to its high and low range over a specific period. It is derived from the RSI but provides more sensitivity and a higher frequency of trading signals by applying the stochastic oscillator formula to RSI values.

$$ \text{StochRSI} = \frac{\text{RSI} - \text{Min(RSI, n)}}{\text{Max(RSI, n)} - \text{Min(RSI, n)}} $$

where:
- $\text{RSI}$ is the current value of the Relative Strength Index,
- $\text{Min(RSI, n)}$ is the minimum RSI value over the last $n$ periods,
- $\text{Max(RSI, n)}$ is the maximum RSI value over the last $n$ periods,
- $n$ is the specified period over which the highs and lows are calculated.

In [ ]:
#min and max RSI values over window
min_rsi = df['RSI'].rolling(window=20).min()
max_rsi = df['RSI'].rolling(window=20).max()

#calc Stochastic RSI
df['Stochastic_RSI'] = (df['RSI'] - min_rsi) / (max_rsi - min_rsi)

In [ ]:
#scale to 0-100 range (better for interpretation and comparison to other oscillators)
df['Stochastic_RSI'] = df['Stochastic_RSI'] * 100

### Awesome Oscillator

The awesome oscillator is a market momentum indicator that compares the recent market momentum, with the momentum over a broader timeframe, by calculating the difference between a 34 period and a 5 period simple moving averages (SMA) of the median prices (or mid prices when median prices are not available).
Calc by-

$$
AO = SMA_{5} - SMA_{34}
$$

where
- $SMA_{5}$ is the 5-period simple moving average of the mid-prices.
- $SMA_{34}$ is the 34-period simple moving average of the mid-prices.

The AO serves to detect changes in the market's momentum and potentially signal upcoming market reversals.

In [ ]:
#calculate sma for 34 and 5 periods
sma_34 = df['Mid_Price'].rolling(window=34, min_periods=1).mean()
sma_5 = df['Mid_Price'].rolling(window=5, min_periods=1).mean()

#calc ao
df['Awesome_Oscillator'] = sma_5 - sma_34

### Accelerator Oscillator

The accelerator oscillator measures the acceleration or deceleration of the current market driving force, essentially indicating whether the market force is increasing or decreasing.

Its calculated as the difference between the 5-period simple moving average of the awesome oscillator, and the ao itself:

$$
AC = AO - SMA_{5}(AO)
$$

where-
- $AO$ is the awesome oscillator.
- $SMA_{5}(AO)$ is the 5-period simple moving average of the awesome oscillator values.

It can be used to confirm ao signals or predict possible reversals.

In [ ]:
ao_sma_5 = df['Awesome_Oscillator'].rolling(window=5, min_periods=1).mean()
df['Accelerator_Oscillator'] = df['Awesome_Oscillator'] - ao_sma_5

### Moving Average Convergence/Divergence Oscillator

MACD is a trend-following momentum indicator that shows the relationship between two moving averages of a security’s price.

$$
\text{MACD} = EMA_{12}(\text{price}) - EMA_{26}(\text{price})
$$
$$
\text{Signal Line} = EMA_{9}(\text{MACD})
$$

where
- $EMA_{12}$ and $EMA_{26}$ are the exponential moving averages for 12 and 26 periods, respectively.
- The Signal Line is the exponential moving average of the MACD itself.

In [ ]:
#calc macdd and signal

#12 and 26 industry standards
ema_12 = df['Mid_Price'].ewm(span=12, adjust=False).mean()
ema_26 = df['Mid_Price'].ewm(span=26, adjust=False).mean()

df['MACD'] = ema_12 - ema_26
df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

### Hull Moving Average

Hull ma reduces the lag of traditional moving averages with improved smoothing and responsiveness.

Calc a weighted moving average with period half the length of the Hull MA, then calculate a WMA for the full period of the Hull MA and subtract it from the first WMA calculation, and finally, calculate a WMA of the result with a period the square root of the Hull MA length.

$$
\text{Hull MA} = WMA(2 * \text{WMA}(n/2) - \text{WMA}(n), \sqrt{n})
$$

where
- $WMA(n)$ is the weighted moving average over $n$ periods.
- $\sqrt{n}$ is the square root of the period $n$.

In [ ]:
#calc
import numpy as np

hull_ma_period = int(np.sqrt(9)) #9 isn't industry standard just suggested starting point- may want to experiment here

wma_9 = df['Mid_Price'].rolling(window=9).apply(lambda x: np.dot(x, np.arange(1, 10)) / np.sum(
    np.arange(1, 10)), raw=True)

df['Hull_MA'] = wma_9.rolling(window=hull_ma_period).mean()

### Keltner Channels

Keltner channels are volatility based envelopes set above and below an exponential moving average.

$$
\text{Middle Line} = EMA_{20}(\text{price})
$$
$$
\text{Upper Channel Line} = \text{Middle Line} + 2 \times ATR_{20}
$$
$$
\text{Lower Channel Line} = \text{Middle Line} - 2 \times ATR_{20}
$$

where
- $EMA_{20}$ is the 20-period exponential moving average.
- $ATR_{20}$ is the 20-period average true range.

In [ ]:
#calc
ema_20 = df['Mid_Price'].ewm(span=20, adjust=False).mean()

atr = df['Mid_Price'].rolling(window=20).apply(lambda x: np.max(x) - np.min(x), raw=True)

df['Keltner_Channel_Middle'] = ema_20
df['Keltner_Channel_Upper'] = ema_20 + 2 * atr
df['Keltner_Channel_Lower'] = ema_20 - 2 * atr

### Detrended Price Oscillator

Detrended price oscillator is an indicator designed to remove the trend from price and allow the measurement of the length and magnitude of price cycles from peak to peak or trough to trough.

DPO is calculated by subtracting the displaced moving average from the price \( \frac{lookback\ period}{2} + 1 \) periods ago.

$$
\text{DPO} = P_{t - \left(\frac{\text{lookback period}}{2} + 1\right)} - SMA_{t - \left(\frac{\text{lookback period}}{2}\right)}
$$

where
- \( P_{t} \) is the price at time \( t \).
- \( SMA \) is the simple moving average over the lookback period.
- The lookback period is the number of periods used to calculate the SMA and displace it.

By removing trends from the price data, the DPO helps to identify cycles and overbought or oversold conditions.

In [ ]:
#efine lookback period
lookback_period = 20

#calc sma
df['SMA_20'] = df['Mid_Price'].rolling(window=lookback_period).mean() #may already have this from noise suppression?

#calc detrended 
#shift sma backwards by (lookback_period / 2 + 1) periods
df['DPO'] = df['Mid_Price'].shift(int(lookback_period / 2 + 1)) - df['SMA_20'].shift(int(lookback_period / 2))

### Bollinger Bands

- Bollinger Bands- moveing average calc over specified period e.g. 20 days. + bands for moving average + or - 2 Standard deviations. (Could be 1.5 or 2.5) 1.9-2.2 is usual for algorithmic trading. Identify volatility and potential trend reversals. When prices are near the upper band the asset may be overbought indicating a possible reversal or pullback. When prices are near the lower band- asset may be oversold suggesting a upward reversal.

In [ ]:
#bollinger bands
MA = df['Mid_Price'].rolling(window=20).mean() #over 20 periods
SD = df['Mid_Price'].rolling(window=20).std()

df['Upper_BB'] = MA + (2 * SD) #upper bb
df['Lower_BB'] = MA - (2 * SD) #lower bb

In [ ]:
#plot
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="darkgrid")
plt.figure(figsize=(12, 6))

plt.plot(df.index, df['Mid_Price'], label='Price', color='blue')
plt.plot(df.index, df['Upper_BB'], label='Upper Bollinger Band', linestyle='--', color='red')
plt.plot(df.index, df['Lower_BB'], label='Lower Bollinger Band', linestyle='--', color='green')

#truncate to see pattern
start_index = 16500  
end_index = 17500  
plt.xlim(start_index, end_index)

plt.xlabel('Date')
plt.ylabel('Mid_Price')
plt.title('Mid price and bollinger bands')
plt.legend()

plt.show()

### Realised Semi Variance

Realised semi-variance is a measure of the downside risk of an asset's returns. It's similar to realised variance but considers only the returns that fall below a certain threshold (typically, returns less than zero, focusing on negative returns). Realised semi-variance provides insight into the volatility resulting from negative market movements, which is useful for risk management purposes.

Calculated by-

$$
\text{Realised Semi-Variance} = \sum_{i=1}^{n} \min(r_i, 0)^2
$$

where $r_i$ represents the log return at interval $i$, and $n$ is the total number of intervals.

I will need to create this over a rolling window so it can apture how downside volatility changing over time. 

In [ ]:
#calc log returns (natural log of ratio of its final price to its initial price)
df['Log_Returns'] = np.log(df['Mid_Price'] / df['Mid_Price'].shift(1))

#define window size 
window_size = 20 #choosing 20 to be consistent with bollinger bands 

#calculate realised semi-variance for each window
df['Realised_Semi_Variance'] = df['Log_Returns'].rolling(window=window_size, min_periods=1).apply(lambda x: np.sum(
    np.minimum(x, 0)**2), raw=True)

In [ ]:
nan_count = df['Log_Returns'].isna().sum()

print(f"Number of NaN values in 'Log_Returns': {nan_count}")

In [ ]:
df = df.dropna(subset=['Log_Returns'])

### Realised Volatility

Realised volatility measures the variation in prices over a period and is a common measure of financial market volatility. For a rolling window calculation, we use log returns and square them to represent variance, which underpins volatility. The realised volatility over the rolling window is the square root of the rolling sum of squared log returns

This calculation provides a dynamic measure of volatility that updates over time, capturing the changing risk profile of the asset.

In [ ]:
#def rolling window
window_size = 20

#calc squared log returns
df['Squared_Log_Returns'] = df['Log_Returns'] ** 2

#calc rolling realised volatility
df['Realised_Volatility'] = np.sqrt(df['Squared_Log_Returns'].rolling(window=window_size).sum())

print((df['Squared_Log_Returns'] < 0).sum())

### Realised Bipower Variation

Realised bipower variation is a measure used to estimate the variance of financial asset returns, focusing specifically on the continuous component of price movements and excluding jumps. It is particularly useful in high-frequency finance for analysing volatility under normal market conditions. Calculated as-

$$
RBV = \frac{\pi}{2} \cdot \sum_{i=1}^{n-1} |r_i| \cdot |r_{i+1}|
$$

where:
- $|r_i|$ is the absolute value of the log return at interval $i$,
- $n$ is the total number of log returns in the series.

RBV isolates the continuous part of price movements by leveraging the properties of bipower measures, which provides us with a robust estimate of volatility that excludes large, discontinuous jumps.

In [ ]:
#calc absolute log returns
df['Abs_Log_Returns'] = df['Log_Returns'].abs()

#calc realised bipower variation
#using .shift(-1) to get r_{i+1} for each r_i, multiplying adjacent abs returns
df['Realised_Bipower_Variation'] = (np.pi / 2) * (df['Abs_Log_Returns'] * df['Abs_Log_Returns'].shift(
    -1)).rolling(window=2).sum()

### Jump Variation

Jump variation is a measure used to isolate the discontinuous (jump) components in asset prices, distinguishing them from the continuous components typically captured by other volatility measures. This distinction is particularly relevant in financial markets where sudden price movements can have significant implications.


**Total quadratic variation** ($TQV$) encompasses both continuous movements and jumps in the asset's price. It is calculated as the sum of squared log returns over a specific period.

$$TQV = \sum_{i=1}^{n} r_i^2$$
   where $r_i$ is the log return at interval $i$.


**Realised bipower variation** ($RBV$) estimates the continuous component of price movements, excluding jumps, by considering the product of absolute log returns for adjacent periods.

$RBV = \frac{\pi}{2} \cdot \sum_{i=1}^{n-1} |r_i| \cdot |r_{i+1}|$$

This uses adjacent pairs of absolute log returns to approximate the continuous component of volatility.

**Realised bipower variation** ($JV$) isolates the jump component by subtracting the continuous component ($RBV$) from the total variation ($TQV$).

$$JV = TQV - RBV$$

In [ ]:
#squared log returns already calculated

#calc total quadratic variation as the sum of squared log returns
df['Total_Quadratic_Variation'] = df['Squared_Log_Returns'].rolling(window=20).sum()

#calc jump variation
df['Jump_Variation'] = df['Total_Quadratic_Variation'] - df['Realised_Bipower_Variation']

### Spot Volatility

Spot volatility reflects the instantaneous volatility of an asset, capturing the expected level of fluctuation in its price at any given moment. This approach is particularly useful in high-frequency trading and market microstructure analysis, where actual transaction data may be sparse but quote data (bids and asks) are available.

**GARCH Model**
 The GARCH model is then fitted to the log returns. GARCH models are used to estimate time-varying volatility, taking into account the conditional variance of past errors and past variances.
   
A basic GARCH(1,1) model includes one lag of squared residuals and one lag of conditional variance, capturing the clustering of volatility over time.

**Forecasting Spot Volatility**
Using the fitted GARCH model, forecast next periodvolatility to estimate the spot volatility. The forecasted variance ($\hat{\sigma}^2_t$) from the GARCH model provides an estimate of the expected volatility for the next period:

   $$ \text{Spot Volatility} = \sqrt{\hat{\sigma}^2_t} $$

In [ ]:
#log returns already calculated

from arch import arch_model

#fit GARCH(1,1) model
am = arch_model(df['Log_Returns']*100, vol='Garch', p=1, q=1, mean='Zero')
res = am.fit(update_freq=5, disp='off')
             
             
#forecast next period's volatility 
forecasts = res.forecast(horizon=1)

#extract last forecasted variance and take the square root for spot volatility
df['Spot_Volatility'] = np.sqrt(forecasts.variance.iloc[-1] / 100)

In [ ]:
df